In [1]:
pip install fake_useragent pandas

  Using cached fake_useragent-1.5.1-py3-none-any.whl.metadata (15 kB)
  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached fake_useragent-1.5.1-py3-none-any.whl (17 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
Using cached numpy-2.1.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.0 MB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd


In [3]:
url = 'https://www.imdb.com/chart/boxoffice/?ref_=nv_ch_cht'
ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}

page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content,'html.parser')


In [4]:
movie_box = soup.find_all('div', class_ = "ipc-metadata-list-summary-item__c")

titles = []
numbers = []
rate_votes = []

for box in movie_box:

    #titles
    if box.find('h3', class_ = "ipc-title__text") is not None:
        title = box.find('h3', class_ = "ipc-title__text")
        titles.append(title.text)
    else:
        titles.append('None')

    #Numbers
    if box.find('ul', attrs={'data-testid':'title-metadata-box-office-data-container'}) is not None:
        number = box.find('ul', attrs={'data-testid':'title-metadata-box-office-data-container'})
        numbers.append(number.text)
    else:
        numbers.append('None')

    #rate_votes
    if box.find('span', attrs={'data-testid':'ratingGroup--imdb-rating'}) is not None:
        rate_vote = box.find('span', attrs={'data-testid':'ratingGroup--imdb-rating'})
        rate_votes.append(rate_vote.text)
    else:
        rate_votes.append('None')



movie_df = pd.DataFrame({
    'title':titles,
    'number': numbers,
    'rate_vote':rate_votes
})
movie_df


,title,number,rate_vote
0,1. Venom: The Last Dance,Weekend Gross: $16MTotal Gross: $119MWeeks Rel...,6.2 (38K)
1,2. Heretic,Weekend Gross: $11MTotal Gross: $11MWeeks Rele...,7.3 (9.9K)
2,3. The Best Christmas Pageant Ever,Weekend Gross: $11MTotal Gross: $13MWeeks Rele...,7.4 (762)
3,4. The Wild Robot,Weekend Gross: $6.6MTotal Gross: $133MWeeks Re...,8.3 (74K)
4,5. Smile 2,Weekend Gross: $5.1MTotal Gross: $62MWeeks Rel...,7.2 (32K)
5,6. Conclave,Weekend Gross: $4.1MTotal Gross: $23MWeeks Rel...,7.6 (8.3K)
6,7. Anora,Weekend Gross: $2.5MTotal Gross: $8.1MWeeks Re...,8.2 (16K)
7,8. Here,Weekend Gross: $2.4MTotal Gross: $10MWeeks Rel...,6.3 (2.5K)
8,9. We Live in Time,Weekend Gross: $2.2MTotal Gross: $22MWeeks Rel...,7.3 (8.1K)
9,10. Terrifier 3,Weekend Gross: $1.7MTotal Gross: $54MWeeks Rel...,6.5 (30K)


In [5]:
movie_df['number'] = movie_df['number'].str.replace('Weekend Gross: ', '').str.replace('Total Gross: ', '|').str.replace('Weeks Released: ', '|')
movie_df[['weekend gross', 'total gross', 'weeks released']] = movie_df['number'].str.split('|', expand=True)

movie_df[['rating', 'vote_count']] = movie_df['rate_vote'].str.split('(', expand=True)
movie_df['vote_count'] = movie_df['vote_count'].str.replace(')','')
# # Drop the original 'rate_vote'
movie_df.drop(['number','rate_vote'], axis=1, inplace=True)
movie_df.to_csv('movie_data.csv')